# Linear Regression Code Along

This notebook is the reference for the video lecture on the Linear Regression Code Along. Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

In [1]:
import findspark

In [2]:
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7/')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lregression_e_commerce').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('Ecommerce_Customers.csv',header=True,inferSchema = True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [16]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [18]:
##Lets set this data for machine learning
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

### we are only working with numerical data
We are considering these as features:
* Avg Session Length
* Time on App
* Time on Website
* Length of Membership

The label is: as we will predict this for test data
* Yearly Amount Spent

In [20]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [22]:
assembler = VectorAssembler(inputCols = ['Avg Session Length',
                                         'Time on App',
                                         'Time on Website',
                                         'Length of Membership'],
                           outputCol='features')

In [27]:
## now we have to transform the data to add features column
output = assembler.transform(data)

In [32]:
output.select('features').show()
## Thats how spark wants to see data for ML

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [33]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [34]:
final_data = output.select('features','Yearly Amount Spent')

In [35]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [36]:
## lets split the data for train and test
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [44]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                362|
|   mean|  498.7137198327258|
| stddev|  79.68395159772662|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [45]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                138|
|   mean|  500.8887865931068|
| stddev|  78.60427882529419|
|    min|  282.4712457199145|
|    max|  765.5184619388373|
+-------+-------------------+



In [50]:
lr = LinearRegression(featuresCol='features',labelCol='Yearly Amount Spent')

In [55]:
lr_model = lr.fit(train_data)

In [56]:
test_results = lr_model.evaluate(test_data)

In [57]:
test_results.rootMeanSquaredError
## Here you can see there is only error of 9.8$ 

9.808492447509765

In [62]:
train_results = lr_model.evaluate(train_data)

In [63]:
train_results.rootMeanSquaredError

9.994024514026133

In [64]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  10.96541306266522|
|-11.835375901075281|
| -5.626122843088808|
|   5.15407110440799|
| 10.350074481114461|
| -3.970092314944168|
|-3.1360448645357337|
| -16.94473424379248|
| 18.478163386052984|
|  7.306532201291873|
|-25.812235476577712|
|  -5.83281698720441|
| -5.759240113503154|
| 2.5976540849340495|
|  1.045638706095815|
| -5.706502786555518|
|-16.671736942621976|
| -2.026225460758212|
| 11.972151570670633|
|  5.883347102028324|
+-------------------+
only showing top 20 rows



In [66]:
test_results.r2

0.9843155003719847

In [67]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [68]:
unlabeled_data = test_data.select('features')

In [69]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[30.4925366965402...|
|[31.0472221394875...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.4252268808548...|
|[31.5702008293202...|
|[31.6005122003032...|
|[31.6548096756927...|
|[31.6739155032749...|
|[31.7207699002873...|
|[31.7242025238451...|
|[31.7366356860502...|
|[31.8293464559211...|
|[31.8745516945853...|
|[31.9048571310136...|
|[31.9120759292006...|
|[31.9262720263601...|
|[31.9480174211613...|
+--------------------+
only showing top 20 rows



In [73]:
##Prediction on unlabeled data
predictions = lr_model.transform(unlabeled_data)

In [72]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|397.67493800996226|
|[30.3931845423455...| 331.7642457042689|
|[30.4925366965402...|288.09736856300333|
|[31.0472221394875...| 387.3433280846134|
|[31.1695067987115...|417.00645632117835|
|[31.2681042107507...| 427.4406254887681|
|[31.4252268808548...| 533.9027635192977|
|[31.5702008293202...| 562.8902263851974|
|[31.6005122003032...|460.69468810504395|
|[31.6548096756927...|467.95689152625664|
|[31.6739155032749...| 501.5373033864589|
|[31.7207699002873...| 544.6077504652274|
|[31.7242025238451...|509.14712740146365|
|[31.7366356860502...|494.33579217059787|
|[31.8293464559211...| 384.1066992818792|
|[31.8745516945853...|  397.991747032823|
|[31.9048571310136...| 490.6215943654381|
|[31.9120759292006...|389.56094176646593|
|[31.9262720263601...|380.23278187365577|
|[31.9480174211613...| 456.0375297908695|
+--------------------+------------